In [12]:
import math

first_frame = 159 # first frame that the blower leaves
last_frame = 617 # last frame just before blower hits wall

height_body = 1.295 # height of blower in meters
radius_body = 0.94 / 2 # radius of blower in meters

height_top = 0.147 # height of top of blower in meters
radius_top = 0.474 / 2 # radius of top of blower in meters

total_time = (last_frame - first_frame) / 30 # total time in seconds
max_speed = 43 / 2.237 # max speed of animal converted from mph to m/s

In [13]:
volume_body = math.pi * (radius_body**2) * height_body # volume of blower in m^3
volume_air = math.pi * ((radius_body - 0.04)**2) * (height_body - 0.04) # volume of air in m^3

volume_top = math.pi * (radius_top**2) * height_top # volume of top of blower in m^3

volume_aluminium = volume_body - volume_air + volume_top # volume of aluminium in m^3

density_aluminium = 2700 # density of aluminium in kg/m^3
mass_aluminium = volume_aluminium * density_aluminium # mass of aluminium in kg

density_air = 1.293 # density of air in kg/m^3
mass_air = volume_air * density_air # mass of air in kg

In [14]:
weight = mass_aluminium + mass_air # weight of blower in kg
normal_force = 9.81 * weight

roll_friction_coefficient = 0.01 # friction coefficient of the ground with the wheels

ground_friction_newton = roll_friction_coefficient * normal_force

frontal_area = 2 * math.pi * radius_body * height_body / 2 # frontal area of blower in m^2

max_air_friction_newton = 0.5 * density_air * (max_speed ** 2) * 0.47 * frontal_area # air friction in newton at max speed
total_power_newton = ground_friction_newton + max_air_friction_newton # total friction in newton at max speed

In [15]:
from plotly.offline import init_notebook_mode
import plotly.graph_objects as go

init_notebook_mode(connected=True)

In [16]:
total_frames = last_frame - first_frame # total frames. framerate is 30fps
delta_t = 1/30

velocity = 0
distance = 0

time_data = []
velocity_data = []
distance_data = []

def get_acceleration(current_speed: float) -> float:
    air_friction_newton = 0.5 * density_air * (current_speed ** 2) * 0.47 * frontal_area # air friction in newton
    total_friction_newton = ground_friction_newton + air_friction_newton # total friction in newton
    
    return (total_power_newton - total_friction_newton) / weight # netto F gedeeld door m is versnelling


for i in range(total_frames):
    acceleration = get_acceleration(velocity) # acceleration in m/s^2
    velocity += acceleration * delta_t
    if velocity > max_speed:
        velocity = max_speed
    
    distance += velocity * delta_t

    velocity_data.append(velocity)
    distance_data.append(distance)
    time_data.append(i * delta_t)
    
velocity_graph = go.Scatter(
    x=time_data,
    y=velocity_data,
    name="Snelhijd stofzuiger"
)    
    
distance_graph = go.Scatter(
    x=time_data,
    y=distance_data,
    name="Afstand stofzuiger"
)

velocity_fig = go.Figure(velocity_graph, layout=dict(
    title = "Snelheid van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Snelheid (m/s)",
    autosize=False
))

distance_fig = go.Figure(distance_graph, layout=dict(
    title = "Afstand van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Afstand (m)",
    autosize=False
))

velocity_fig.show()
distance_fig.show()